In [ ]:
## Hu clustering 

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import scvelo as scv

In [ ]:
sc.settings.verbosity=3
sc.logging.print_header()

In [ ]:
hu = sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/1c/hu_raw.h5ad') 
hu.obs['Patient']= hu.obs['Patient'].astype('category')

In [ ]:
hu.var_names_make_unique()
sc.external.pp.scrublet(hu)
sc.pl.highest_expr_genes(hu, n_top=20, )

In [ ]:
sc.pp.filter_cells(hu, min_genes=200)
sc.pp.filter_genes(hu, min_cells=3)
hu.var['mt']=hu.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(hu, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(hu, ['n_genes_by_counts','total_counts','pct_counts_mt'],jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(hu, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(hu, x='total_counts', y='n_genes_by_counts')

In [ ]:
hu=hu[hu.obs.n_genes_by_counts <5000,:]
hu=hu[hu.obs.total_counts <1000000,:]
hu=hu[hu.obs.pct_counts_mt <30,:]
hu.shape

In [ ]:
sc.pp.normalize_total(hu, target_sum=1e4) 
sc.pp.log1p(hu)
sc.pp.highly_variable_genes(hu, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(hu)
hu.raw = hu
hu = hu [:, hu.var.highly_variable]
sc.pp.regress_out(hu, ['total_counts','pct_counts_mt'])
sc.pp.scale(hu, max_value=10)

In [ ]:
sc.tl.pca(hu, svd_solver='arpack')
sc.pl.pca(hu, color='OVGP1')
sc.pl.pca_variance_ratio(hu, log=True)
sc.pp.neighbors(hu, n_neighbors=10, n_pcs=8)
sc.tl.umap(hu)
sc.tl.leiden(hu, resolution=0.4)
sc.pl.umap(hu, color=['OVGP1','SNTN'])

In [ ]:
sc.pl.umap(hu, color=['leiden','OVGP1','SNTN'])

In [ ]:
sc.tl.rank_genes_groups(hu,'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(hu, n_genes=25, sharey=False)

In [ ]:
cell_dict = {'Secretory': ['0','2','3','4','8'], 
 'Fibroblast': ['6'],
 'Ciliated': ['1','5'], 
'Leukocytes': ['7'],
 'Endothelial': ['9'],}

hu.obs['Cell_Types'] = np.nan

for i in cell_dict.keys():
    ind = pd.Series(hu.obs.leiden).isin(cell_dict[i])
    hu.obs.loc[ind,'Cell_Types'] = i

In [ ]:
sc.pl.umap(hu, color=['Cell_Types'], legend_loc='right margin', legend_fontsize=12, save= 'huumap.png')

In [ ]:
results_file = ('/home/j87832lw/mounting/oviduct/data/1c/hu_clustering_renamedleidengroups.h5ad')
hu.write(results_file)